## Import libraries

In [ ]:
from distortionStudy import *
from TTreeHnInteractive.TTreeHnBrowser import *
from TTreeHnInteractive.bokehTools import *
import matplotlib.pyplot as plt
from MLpipeline.NDFunctionInterface import PrepareData, Fitter
#matplotlib inline
output_notebook()
#matplotlib notebook

### Load csv file with disortion and split them per sector to enable correlation

In [ ]:
input=os.path.expandvars("$NOTESData/JIRA/ATO-336/DistortionsTimeSeries/distortionAll.csv")
df=readDataFrame(input)
dfsplit=splitDistortionFrame(df)
print("load csv file", input, df.shape, dfsplit.shape)
dfsplit.head(3)
dfsplit=dfsplit.query("iz2x>-1")

### Load fitter and fit distortion in regions as function of other distortions

In [ ]:
deltaColumns=[col for col in dfsplit.columns if "drphiSec" in col ]
factorColumns= ['bz','bckg0Mean','iz2x']
variableX=[x for x in deltaColumns if x != 'drphiSector2']
variableX+=factorColumns
#
x = PrepareData(dfsplit, variableX, ['drphiSector2'], [500,500])
fitter = Fitter(x)
fitter.Register_Method('KM','KerasModel', 'Regressor')
fitter.Register_Method('KNN','KNeighbors', 'Regressor')
fitter.Register_Method('RF','RandomForest', 'Regressor', n_estimators=100, max_depth=10)
fitter.Register_Method('RF200','RandomForest', 'Regressor', n_estimators=200, max_depth=10)
list(variableX)

In [ ]:
fitter.Fit()
fitter.Compress('KM')
for method in ['RF', 'KNN', 'RF200','KM']: 
    dfsplit = fitter.AppendOtherPandas(method,dfsplit)
                                    

In [ ]:
fitter.AppendOtherPandas?
dfsplit.head()

## Bokeh corelation visualization

In [ ]:
p = figure(plot_width=600, plot_height=600, title="s2s4")
plot=drawColz(dfsplit," trdIntMedianL0<30 ","drphiSector2", "RF200","year",p)

In [ ]:
fitter.Models[1].model.feature_importances_

In [ ]:
fitter.Models[1].model

In [ ]:
list(dfsplit)

In [ ]:
#dfsplit.plot.scatter('drphi2','drphis4',c="drphis6")")
p = figure(plot_width=600, plot_height=600, title="s2s4")
plot=drawColz(dfsplit,"bz>0 & iz2x==4& trdIntMedianL0<30 &year==2016","drphi2", "drphi4","trdIntMedianL0",p)

## Playing with pandas - TO BE DELETED later

In [ ]:
list(dfsplit)

In [ ]:
df['fillcolor']=df['fill']-4000
df.query("sector==4&isIROC&year<2017 & abs(trdIntMedianL0/trdIntMedianL1-1)<0.4").plot.scatter('eventsPerSecTPC','trdIntMedianL0',c="fillcolor")

In [ ]:
df['ratio'] = df.trdIntMedianL5 / df.trdIntMedianL0
df['ratiom']=df.drphiSmoothedQ98/df.trdIntMedianL5
fig=drawColz(df,"sector==4&isIROC&iz2x==1&bz>0 &abs(ratio-1)<0.9&year==2016","gascompCO2","ratiom","gascompCO2")

In [ ]:
fig=drawColz(df,"sector==9&isIROC&iz2x==1&bz>0 &abs(ratio-0.8)<0.1&year==2016","trdIntMedianL0","trdIntMedianL5","gascompH2O")

In [ ]:
df1.query("abs(ratio-0.8)<0.2").hist('ratio')

In [ ]:
a=1
"xxx"+str(a)
query="isIROC&isec="+str(a)
query

In [ ]:
sectors=[2,4,6,7,9,15,20,30]
pandaList=[]
dfall=df.query("isec==2&isIROC").reset_index(drop=True)
pandaList=[dfall]
for isector in sectors:
    global pandaList
    query="isIROC&isec=="+str(isector)
    dfsec=df.query(query)[['drphiSmoothedQ95']].reset_index(drop=True)
    dfsec.columns=["drphi"+str(isector)]
    pandaList.append(dfsec)
pall=pd.concat(pandaList,axis=1)
    
pall.head(3) 
pall.query("iz2x==4").plot.scatter("drphi2","drphi20")
drawColz(pall,"iz2x==4","drphi2", "drphi4","")

In [ ]:
dflist=[]
dfall=df.query("isec==2&isIROC&iz2x==3")[:5000].copy(True)
df2=df.query("isec==2&isIROC&iz2x==3")[['drphiSmoothedQ95','time']][:5000].copy(True).reset_index(drop=True)
df4=df.query("isec==4&isIROC&iz2x==3")[['drphiSmoothedQ95','time']][:5000].copy(True).reset_index(drop=True)
df6=df.query("isec==6&isIROC&iz2x==3")[['drphiSmoothedQ95','time']][:5000].copy(True).reset_index(drop=True)
df2.columns= ['s2', 'time']
df4.columns= ['s4', 'time']
df6.columns= ['s6', 'time']

m=pd.concat([df2,df4,df6],axis=1)
m.plot.scatter("s2","s4")
m.head(5)

In [ ]:
print(dfall.shape, df2.shape,df4.shape,df6.shape)
dfall=dfall.join(df2,rsuffix='s2')
dfall=dfall.join(df4,rsuffix='s4')
dfall=dfall.join(df6,rsuffix='s6').copy(true)
print(dfall.shape, df2.shape,df4.shape,df6.shape)
#df6

In [ ]:
drawColz(m,"isIROC==1","s2", "s4","bz")

In [ ]:
dfall.plot.scatter('drphiSmoothedQ95s2','drphiSmoothedQ95s2',c="drphiSmoothedQ95s2")

In [ ]:
dfall['drphiSmoothedQ95s2'].head(5)

In [ ]:
list(df)